<img src="https://scontent-sin6-2.xx.fbcdn.net/v/t1.0-9/10568868_4371370818520_2644688925685876233_n.jpg?_nc_eui2=v1%3AAeGrfap77OpgCCLIjBDyyrk38BYkXsJfdmzG6vQmcsmqPNyv_BaocIUwbjvpyAYA3vvZ8Nml8l9w8EAZ7xKDrzJsc3z4FRuMKiaVxRVpGYwOwA&oh=b1b9af086dd3fb2a969320123dce7e2d&oe=5A66F2E3" width="150" height="150">
<center> Ganteng <center>

<img src="https://scontent-sin6-2.xx.fbcdn.net/v/t1.0-9/20663684_1356660797762965_9003853271414511443_n.jpg?_nc_eui2=v1%3AAeEz0x_IFsOHJiXnbnby5Mw5YYyrLxzjGDm2HBl6rVCx8q5ghhqE1AYKBwMCQzv79ANCs_hrJM5QNizc3lvixV43dxRBd0OdXmHmwkKaEDefyg&oh=eb8052efb0959c722f468801f7e891bf&oe=5A7EE7CD" width="150" height="150">
<center> Ganteng <center>

# Import Necessary Library

In [ ]:
from sklearn.datasets import load_iris
from sklearn import tree
from sklearn import neural_network
from sklearn import model_selection
from sklearn import metrics
from sklearn.externals import joblib
from pandas import read_csv
from pandas import DataFrame
from copy import deepcopy
import graphviz

# A - Read Dataset

## Read Iris Dataset

In [ ]:
iris = load_iris()
iris.feature_names

## Read Tennis Dataset

In [ ]:
tennis = read_csv("./dataset/tennis.csv")
tennisData = tennis.drop('play', axis=1)
tennisLabel = DataFrame(tennis.play)
#Must be latter changed to numpy array to be used with sklearn

# B - Full Train

## DTL

In [ ]:
clfDTL = tree.DecisionTreeClassifier()
clfDTL.fit(iris.data, iris.target)
dotData = tree.export_graphviz(clfDTL, out_file=None, feature_names=iris.feature_names, 
                               class_names = iris.target_names, filled=True)
graph = graphviz.Source(dotData)
graph

## MLP

In [ ]:
clfMLP = neural_network.MLPClassifier(solver='sgd', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=39, max_iter=2000)
clfMLP.fit(iris.data, iris.target)
#Visualize MLP if possible (unlikely)

# C - Split Train

## Split Data

In [ ]:
irisTrainData, irisTestData, irisTrainLabel, irisTestLabel = model_selection.train_test_split(iris.data, iris.target,
                                                                                              test_size=0.1, random_state=39)

## Train and Test

In [ ]:
clfDTL.fit(irisTrainData, irisTrainLabel)
clfMLP.fit(irisTrainData, irisTrainLabel)
DTLResult = clfDTL.predict(irisTestData)
MLPResult = clfMLP.predict(irisTestData)

## Print Confusion Matrix and Classification Report / Performance

In [ ]:
print(metrics.confusion_matrix(DTLResult, irisTestLabel))
print(metrics.confusion_matrix(MLPResult, irisTestLabel))
print(metrics.classification_report(DTLResult, irisTestLabel))
print(metrics.classification_report(MLPResult, irisTestLabel))

# D - 10-Fold

## Split, Find Best Model From k-Fold, Classify Whole Datasets

In [ ]:
kf = model_selection.KFold(n_splits=10)
maxAccuracyDTL = 0
maxAccuracyMLP = 0
index = 0
for trainIndex, testIndex in kf.split(iris.data):
    #Split
    irisTrainData = iris.data[trainIndex]
    irisTrainLabel = iris.target[trainIndex]
    irisTestData = iris.data[testIndex]
    iristTestLabel = iris.target[testIndex]
    #Train
    clfDTL.fit(irisTrainData, irisTrainLabel)
    clfMLP.fit(irisTrainData, irisTrainLabel)
    #Test
    DTLResult = clfDTL.predict(irisTestData)
    MLPResult = clfMLP.predict(irisTestData)
    #Result
    DTLScore = metrics.accuracy_score(DTLResult,irisTestLabel)
    MLPScore = metrics.accuracy_score(MLPResult,irisTestLabel)
    if (DTLScore > maxAccuracyDTL):
        maxAccuracyDTL = DTLScore
        bestDTL = deepcopy(clfDTL)
    if (MLPScore > maxAccuracyMLP):
        maxAccuracyMLP = MLPScore
        bestMLP = deepcopy(clfMLP)
    print("Fold: " + str(index))
    index += 1
DTLResult = bestDTL.predict(iris.data)
MLPResult = bestMLP.predict(iris.data)

## Print Classification Report / Performance

In [ ]:
print(metrics.classification_report(DTLResult, iris.target))
print(metrics.classification_report(MLPResult, iris.target))

# E - Save Model

In [ ]:
joblib.dump(bestDTL,"model/dtl.pkl")
joblib.dump(bestMLP,"model/mlp.pkl")

# F - Load Model

In [ ]:
clfDTL = joblib.load("model/dtl.pkl")
clfMLP = joblib.load("model/mlp.pkl")

# G - Create New Instance

In [ ]:
instance = [[0.001, 0.001, 4.851, 1.751]]

# H - Classify New Instance

In [ ]:
DTLResult = bestDTL.predict(instance)
print(iris.target_names[DTLResult])
MLPResult = bestMLP.predict(instance)
print(iris.target_names[MLPResult])

# I - Features Analysis

In [16]:
#Sea born
#http://seaborn.pydata.org/tutorial/regression.html#regression-tutorial